In [1]:
!pip install flask
from flask import Flask, request, render_template
import numpy as np
import threading
import webbrowser
import pickle
from datetime import datetime
import re

In [2]:
# Getting a free available port: 
# varname: myPort
from socket import socket

with socket() as s:
    s.bind(('',0))
    #print(s.getsockname()[1])
    myPort= s.getsockname()[1]

#print ("myPort=", myPort)
print (myPort)

56718


In [3]:
# Load the trained model
with open('simple_model.pkl', 'rb') as f:
    model = pickle.load(f)
    
#Load scaler for numeric features  
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
    
# Load the LabelEncoder  for 'carrier_type'
with open('carrier_type_encoder.pkl', 'rb') as f:
    carrier_type_encoder = pickle.load(f)

# Load the LabelEncoder  for 'county_of_injury'
with open('county_injury_encoder.pkl', 'rb') as f:
    county_injury_encoder = pickle.load(f)

# Load the LabelEncoder  for 'district_name'
with open('district_name_encoder.pkl', 'rb') as f:
    district_name_encoder = pickle.load(f)
    
# Load the LabelEncoder  for 'medical_region'
with open('medical_fee_region_encoder.pkl', 'rb') as f:
    medical_fee_region_encoder = pickle.load(f)
    
# Load the LabelEncoder  for 'gender'
with open('gender_encoder.pkl', 'rb') as f:
       gender_encoder = pickle.load(f)


In [4]:
# Define the Flask app
app = Flask(__name__)

# Function to convert the date from tt.mm.jjjj to YYYY-MM-DD format
def convert_to_yyyy_mm_dd(date_str):
    date_obj = datetime.strptime(date_str, "%d.%m.%Y")  # Convert string to datetime
    return date_obj.strftime("%Y-%m-%d")  # Return as YYYY-MM-DD

#Instead of manually encoding each feature individually --> function that transforms multiple features at once
def encode_features(features, encoders):
    """
    Encodes a list of features using the corresponding encoders.
    
    :param features: A dictionary of feature names and their values
    :param encoders: A dictionary of encoders where keys are feature names and values are encoder instances
    :return: A list of encoded features
    """
    encoded_features = []
    for feature_name, value in features.items():
        # Apply the corresponding encoder to the feature
        encoder = encoders.get(feature_name)
        if encoder:
            encoded_value = encoder.transform([value])[0]
            encoded_features.append(encoded_value)
        else:
            # Handle case where no encoder is found
            encoded_features.append(value)  # Just append raw value if no encoding is required
    return encoded_features

def extract_numbers(code):
    match = re.match(r'^(\d+)', code)
    return int(match.group(1)) if match else None

# Add the label_mapping dictionary
label_mapping = {
    0: '1. CANCELLED',
    1: '2. NON-COMP',
    2: '3. MED ONLY',
    3: '4. TEMPORARY',
    4: '5. PPD SCH LOSS',
    5: '6. PPD NSL',
    6: '7. PTD',
    7: '8. DEATH'
}

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Gather form inputs
        accident_date = request.form['accident-date']
        age_at_injury = float(request.form['age-at-injury'])
        alternative_dispute_resolution = request.form['alternative-dispute-resolution']
        assembly_date = request.form['assembly-date']
        attorney_representative = request.form['attorney-representative']
        average_weekly_wage = float(request.form['average-weekly-wage'])
        birth_year = int(request.form['birth-year'])
        c2_date = request.form['c2-date']
        c3_date = request.form['c3-date']
        carrier_type = request.form['carrier-type']
        county_of_injury = request.form['county-of-injury']
        covid19_indicator = request.form['agreement-reached']
        district_name = request.form['district-name']
        first_hearing_date = request.form['first-hearing-date']
        gender = request.form['gender']
        ime4_count = int(request.form['ime4-count'])
        industry_code = request.form['industry-code']
        medical_region = request.form['medical-region']
        wcio_cause_of_injury_code = request.form['wcio-cause-of-injury-code']
        wcio_nature_of_injury_code = request.form['wcio-nature-of-injury-code']
        wcio_part_of_body_code = request.form['wcio-part-of-body-code']
        zip_code = request.form['zip']
        agreement_reached = request.form['agreement-reached']
        wcb_decision = request.form['wcb-decision']

        ###CONVERTING DATES INTO RIGHT FORMAT
        formatted_accident_date = convert_to_yyyy_mm_dd(accident_date)
        formatted_assembly_date = convert_to_yyyy_mm_dd(assembly_date)
        formatted_c2_date = convert_to_yyyy_mm_dd(c2_date)
        formatted_c3_date = convert_to_yyyy_mm_dd(c3_date)
        formatted_first_hearing_date = convert_to_yyyy_mm_dd(first_hearing_date)
        
        formatted_date_features = np.array([[formatted_accident_date, formatted_assembly_date, formatted_c2_date,
                                            formatted_c3_date, formatted_first_hearing_date]])
        
        ###SCALING NUMERICAL FEATURES
        numerical_features = np.array([[age_at_injury, average_weekly_wage, ime4_count]])
        
        # Scale the numerical features using the loaded scaler
        scaled_numerical_features = scaler.transform(numerical_features)

        ###ENCODING BINARY VARIABLES
        # Mapping 'Y'/'yes' to 1, 'N'/'no' to 0
        alternative_dispute_resolution = 1 if feature3 in ['Y', 'yes', 'Yes'] else 0
        attorney_representative = 1 if feature3 in ['Y', 'yes', 'Yes'] else 0
        covid19_indicator = 1 if feature3 in ['Y', 'yes', 'Yes'] else 0
        agreement_reached = 1 if feature3 in ['Y', 'yes', 'Yes'] else 0
        
        binary_features = np.array([[alternative_dispute_resolution, attorney_representative, covid19_indicator,
                                    agreement_reached]])
        
        ##ENCODING ALL CATEGORICAL FEATURES
        # Prepare the features dictionary
        features = {
            'carrier_type': carrier_type,
            'county_of_injury': county_of_injury,
            'district_name': district_name,
            'gender': gender,
            'medical_region': medical_region
        }
        
        # List of encoders for each categorical feature
        encoders = {
            'carrier_type': carrier_type_encoder,
            'county_of_injury': county_injury_encoder,
            'district_name': district_name_encoder,
            'gender': gender_encoder,
            'medical_region': medical_fee_region_encoder
        }
        
        # Encode the features using the helper function
        encoded_features = encode_features(features, encoders)
        
        cat_features = np.array([encoded_features])
        
        # Encode the carrier type
        #carrier_type_encoded = carrier_type_encoder.transform([carrier_type])[0]
        #county_of_injury_encoded = county_injury_encoder.transform([county_of_injury])[0]
        #district_name_encoded = district_name_encoder.transform([district_name])[0]
        #medical_region_encoded = medical_fee_region_encoder.transform([medical_region])[0]
        #carrier_type_encoded = carrier_type_encoder.transform([carrier_type])[0]
        #carrier_type_encoded = carrier_type_encoder.transform([carrier_type])[0]
        #carrier_type_encoded = carrier_type_encoder.transform([carrier_type])[0]
        #carrier_type_encoded = carrier_type_encoder.transform([carrier_type])[0]
        
        ###CODES
        industry_code_number = extract_numbers(industry_code)
        cause_of_injury_number = extract_numbers(wcio_cause_of_injury_code)
        nature_of_injury_number = extract_numbers(wcio_nature_of_injury_code)
        part_of_body_number = extract_numbers(wcio_part_of_body_code)
        
        code_features = np.array([[industry_code_number, cause_of_injury_number, nature_of_injury_number,
                                  part_of_body_number]])
        
        # Combine the scaled features with the categorical feature
        final_features = np.concatenate([scaled_numerical_features, binary_features, cat_features, code_features], axis=1)
        
        # Make the prediction
        prediction = model.predict(final_features)[0]
        
        # Decode the prediction using the label_mapping dictionary
        decision = label_mapping.get(prediction, "Unknown Prediction")

        # Display the result
        return render_template('result.html', decision=decision)
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Function to run Flask in a thread
def run_app():
    app.run('localhost', debug=True, port=myPort, use_reloader=False)

# Start Flask in a thread
threading.Thread(target=run_app).start()

# Open the app in a browser
webbrowser.open(f"http://127.0.0.1:{myPort}")
